In [1]:
import os
import numpy as np
from matplotlib import pyplot
# Modulo de optimización
from scipy import optimize
%matplotlib inline

Marca ,Número de puertas, Número de asientos, Condición del vehículo,Transmisión,Tipo de Combustible,Tipo,"Modelo (Ej: 2021, 2016)",Procedencia,Kilometraje (recorrido en Km),Cilindrada (en cc),Estado de la pintura,¿La pintura está en buen estado?,"¿El vehículo tuvo algún accidente? ","¿El vehículo cuenta con todos los accesorios? ",Costo en $us,datos_falso(1,2)

In [2]:
data_train = np.loadtxt('preparado.csv', delimiter=',', dtype=float)
data_test = np.loadtxt('test.csv', delimiter=',', dtype=float)
#print(data_train)
#print(data_test)

In [3]:
X = data_train[:,0:16]
X_test = data_test[:,0:16]
#print(X_test)

In [4]:
y = np.where(data_train[:,16]==1, 1,0)
y = np.asarray(y, dtype=np.int)
y_test = np.where(data_test[:,16]==1, 1,0)
y_test = np.asarray(y_test, dtype=np.int)
X.shape, y.shape


((1012, 16), (1012,))

In [5]:
#funciondes de activacion
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

In [6]:
def linear(x):
    return x

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

funciones de perdida

In [7]:
# Binary Cross Entropy -> usada para clasificación binaria (con sigmoid)
def bce(y, y_hat):
    return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))

# Cross Entropy (aplica softmax + cross entropy de manera estable) -> usada para clasificación multiclase
def crossentropy(y, y_hat):
    logits = y_hat[np.arange(len(y_hat)),y]
    entropy = - logits + np.log(np.sum(np.exp(y_hat),axis=-1))
    return entropy.mean()

Derivadas

In [8]:
def grad_mse(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_bce(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_crossentropy(y, y_hat):
    answers = np.zeros_like(y_hat)
    answers[np.arange(len(y_hat)),y] = 1    
    return (- answers + softmax(y_hat)) / y_hat.shape[0]

In [9]:
# clase base MLP 

class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2 
    return self.activation(y_hat)
    
  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):     
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size] 
            # salida del perceptrón
            y_pred = self(x) 
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)        
            # Backprop 
            dldy = self.grad_loss(y, y_pred) 
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)      
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)

In [10]:
# MLP para clasificación binaria
class MLPBinaryClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, bce, grad_bce, sigmoid)

# MLP para clasificación multiclase
class MLPClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, crossentropy, grad_crossentropy, linear)

In [11]:
# normalización datos
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_norm = (X - X_mean) / X_std

In [12]:
model = MLPBinaryClassification(D_in=16, H=10, D_out=1)
epochs = 100
lr = 0.001

model.fit(X_norm, y, epochs, lr, batch_size=1, log_each=10)

Epoch: 10/100, Loss: 0.04004
Epoch: 20/100, Loss: -0.00176
Epoch: 30/100, Loss: -0.01889
Epoch: 40/100, Loss: -0.02741
Epoch: 50/100, Loss: -0.03201
Epoch: 60/100, Loss: -0.03457
Epoch: 70/100, Loss: -0.03587
Epoch: 80/100, Loss: -0.03635
Epoch: 90/100, Loss: -0.03639
Epoch: 100/100, Loss: -0.03620


In [13]:
model2 = MLPClassification(D_in=16, H=100, D_out=2)
epochs = 1000
lr = 0.02
model2.fit(X_norm, y, epochs, lr, batch_size=10, log_each=100)

Epoch: 100/1000, Loss: 0.15409
Epoch: 200/1000, Loss: 0.11952
Epoch: 300/1000, Loss: 0.10043
Epoch: 400/1000, Loss: 0.08529
Epoch: 500/1000, Loss: 0.07368
Epoch: 600/1000, Loss: 0.06508
Epoch: 700/1000, Loss: 0.05853
Epoch: 800/1000, Loss: 0.05342
Epoch: 900/1000, Loss: 0.04933
Epoch: 1000/1000, Loss: 0.04598


modelo Uno con datos de prueba

In [14]:
w=model.ws[99]
X_norm_test1 = (X_test - X_mean) / X_std
y_predict = model.predict(w, X_norm_test1)
for i in y_predict:
    print(np.argmax(i))

0
0
0
0
0
0
0
0
0
0
0


Modelo 2 con datos de prueba

In [15]:
X_norm_test = (X_test - X_mean) / X_std
y_predict = model2.predict(model2.ws[999], X_norm_test)
y_predict_test=[]
for i in y_predict:
    y_predict_test.append(np.argmax(i))
    print(np.argmax(i))

1
1
1
0
1
0
0
0
0
0
1


In [16]:
y_test

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])

In [17]:
y_predict_ = model2.predict(model2.ws[999], X_norm)
y_pred_train=[]
for i in y_predict_:
    y_pred_train.append(np.argmax(i))
    #print(np.argmax(i))

In [18]:
def accuracy(y_pred, y):
    return np.sum(y_pred == y) / len(y)

precicion con datos te entrenamiento

In [19]:
accuracy(y_pred_train, y)

0.9950592885375494

In [20]:
accuracy(y_predict_test,y_test)

0.8181818181818182